In [1]:
import numpy as np
import pprint

### 8.1 a

In [2]:
def readTitles():
    return [line.rstrip() for line in open("hw8_movieTitles.txt").readlines()]

def readPIDs():
    return [line.rstrip() for line in open("hw8_studentPIDs.txt").readlines()]

def readRatings():
    return [[int(x) if x != '?' else -1 for x in line.rstrip().split(" ")] 
            for line in open("hw8_ratings.txt").readlines()]

def orderByPopularity():
    meanratingofmovies = [[el for el in x if el != -1] for x in zip(*ratings)]
    meanratingofmovies = [sum(x)/len(x) for x in meanratingofmovies]
    
    movieswithrating = [[movie, rating] for (movie, rating) 
                        in zip(movies, meanratingofmovies)]
    movieswithrating.sort(key=lambda x: x[1])
    return movieswithrating

movies = readTitles()
students = readPIDs()
ratings = readRatings()
movieratings = orderByPopularity()

In [3]:
pprint.pprint(movieratings)

[['Fifty_Shades_of_Grey', 0.33093525179856115],
 ['The_Last_Airbender', 0.35714285714285715],
 ['Magic_Mike', 0.5245901639344263],
 ['Prometheus', 0.5436893203883495],
 ['Bridemaids', 0.5555555555555556],
 ['World_War_Z', 0.59],
 ['Man_of_Steel', 0.5915492957746479],
 ['Mad_Max:_Fury_Road', 0.6333333333333333],
 ['Drive', 0.6363636363636364],
 ['Thor', 0.6646341463414634],
 ['Pitch_Perfect', 0.6666666666666666],
 ['The_Hunger_Games', 0.6712962962962963],
 ['Fast_Five', 0.6783216783216783],
 ['The_Hateful_Eight', 0.6825396825396826],
 ['Iron_Man_2', 0.6923076923076923],
 ['The_Perks_of_Being_a_Wallflower', 0.6951219512195121],
 ['American_Hustle', 0.7159090909090909],
 ['The_Help', 0.725],
 ['Avengers:_Age_of_Ultron', 0.7298850574712644],
 ['21_Jump_Street', 0.7358490566037735],
 ['Captain_America:_The_First_Avenger', 0.7360406091370558],
 ['Les_Miserables', 0.7452830188679245],
 ['Star_Wars:_The_Force_Awakens', 0.7485714285714286],
 ['Jurassic_World', 0.7540106951871658],
 ['The_Great_

### 8.1 e

In [4]:
def initz():
    return [float(line.rstrip()) for line 
            in open("hw8_probZ_init.txt").readlines()]
        
def initrgivenz():
    return [[float(el) for el in line.rstrip().split()] 
            for line in open("hw8_probRgivenZ_init.txt").readlines()]

z = initz()
rgivenz = initrgivenz()
#pprint.pprint(rgivenz)
#pprint.pprint(ratings[0])
#print(z)
def Estep(z, rgivenz, ratings):
    rho = []*len(ratings)
    for t in range(len(ratings)):
        rho.append([1]*len(z))
        
    for t in range(len(ratings)):
        den = 0
        for i in range(len(z)):
            rho[t][i] *= z[i]
            for j in range(len(rgivenz)):
                if ratings[t][j] == 1:
                    rho[t][i] *= rgivenz[j][i]
                elif ratings[t][j] == 0:
                    rho[t][i] *= (1 - rgivenz[j][i])

            den += rho[t][i]

        for i in range(len(z)):
            rho[t][i] /= den

    return rho

def Mstep(z, rgivenz, ratings, rho):
    samplecount = len(ratings)
    newz = [0]*len(z)
    newrgivenz = []*len(rgivenz)
    for j in range(len(rgivenz)):
        newrgivenz.append([0]*len(z))
    
    for i in range(len(z)):
        newz[i] = sum([x[i] for x in rho])/samplecount
            
    for t in range(samplecount):
        for i in range(len(z)):
            for j in range(len(rgivenz)):
                if ratings[t][j] != -1:
                    if ratings[t][j] == 1:
                        newrgivenz[j][i] += rho[t][i]
                else:
                    newrgivenz[j][i] += rho[t][i] * rgivenz[j][i]
        
    for j in range(len(rgivenz)):
        for i in range(len(z)):
            newrgivenz[j][i] /= sum([x[i] for x in rho])
            
#     print(newz)
#     print(newrgivenz)
    return (newz, newrgivenz)
    

def likelihood(z, rgivenz, ratings):
    samplecount = len(ratings)
    ll = 0
    for t in range(len(ratings)):
        val = 0
        for i in range(len(z)):
            prob = 1
            for j in range(len(rgivenz)):
                if ratings[t][j] != -1:
                    if ratings[t][j] == 1:
                        prob *= rgivenz[j][i]
                    else:
                        prob *= (1 - rgivenz[j][i])                       

            val += prob*z[i]
        ll += np.log(val)/samplecount
    return ll
                    
            
def EM(iters, z, rgivenz, ratings):
    for i in range(iters):
        ll = likelihood(z, rgivenz, ratings)
        if i in [0,1,2,4,8,16,32,64]:
            print("Log Likelihood at Iteration {}: {}"
                  .format(i, likelihood(z, rgivenz, ratings)))
        rho = Estep(z, rgivenz, ratings)
#         pprint.pprint(rho)
        z, rgivenz = Mstep(z, rgivenz, ratings, rho)
    print("Log Likelihood at Iteration {}: {}"
          .format(i+1, likelihood(z, rgivenz, ratings)))
    return (rho, z, rgivenz)

rho, z, rgivenz = EM(64, z, rgivenz, ratings)

Log Likelihood at Iteration 0: -23.68194303920655
Log Likelihood at Iteration 1: -14.342138732059835
Log Likelihood at Iteration 2: -12.909592229693583
Log Likelihood at Iteration 4: -12.150619817940063
Log Likelihood at Iteration 8: -11.867860888836633
Log Likelihood at Iteration 16: -11.682204313006864
Log Likelihood at Iteration 32: -11.565450274671571
Log Likelihood at Iteration 64: -11.540129139078093


### 8.1 f

In [7]:
myratings = ratings[students.index("A53247488")]
myratingindex = students.index("A53247488")
expected_ratings = [0]*len(myratings)
for j in range(len(myratings)):
    for i in range(len(z)):
        if myratings[j] == -1:
            expected_ratings[j] += rho[myratingindex][i] * rgivenz[j][i]
            
suggested_movies = [[i,val] for i,val 
                    in enumerate(expected_ratings) if val > 0]
suggested_movies.sort(key=lambda x: x[1], reverse=True)
[movies[i] for i, val in suggested_movies]

['The_Dark_Knight_Rises',
 'Gone_Girl',
 'The_Social_Network',
 'The_Revenant',
 '12_Years_a_Slave',
 'The_Theory_of_Everything',
 'Les_Miserables',
 '21_Jump_Street',
 'Django_Unchained',
 'Captain_America:_The_First_Avenger',
 'Avengers:_Age_of_Ultron',
 'Pitch_Perfect',
 'Ex_Machina',
 'The_Help',
 'The_Great_Gatsby',
 'Star_Wars:_The_Force_Awakens',
 'The_Girls_with_the_Dragon_Tattoo',
 'Wolf_of_Wall_Street',
 'Room',
 'Her',
 'Midnight_in_Paris',
 'The_Hateful_Eight',
 'Mad_Max:_Fury_Road',
 'Fast_Five',
 'The_Perks_of_Being_a_Wallflower',
 'American_Hustle',
 'World_War_Z',
 'Bridemaids',
 'Man_of_Steel',
 'Magic_Mike',
 'Drive',
 'The_Last_Airbender',
 'Fifty_Shades_of_Grey']